In [1]:
!pip install lightfm

In [2]:
import pickle
from lightfm.data import Dataset
import numpy as np
import pandas as pd
from collections import OrderedDict

with open('recsys.pickle', 'rb') as f:
   model = pickle.load(f)

In [3]:
pst1_df = pd.read_csv('emj_df1.csv')
pst2_df = pd.read_csv('emj_df2.csv')
clean_pst_df = pd.concat([pst1_df, pst2_df], axis=0)
clean_pst_df = clean_pst_df.reset_index(drop=True)
del pst1_df
del pst2_df
pst3_df = pd.read_csv('emj_df3.csv')
pst4_df = pd.read_csv('emj_df4.csv')
clean_pst_df = pd.concat([clean_pst_df, pst3_df, pst4_df], axis=0)
clean_pst_df = clean_pst_df.reset_index(drop=True)
del pst3_df
del pst4_df

In [4]:
train_data = clean_pst_df[['user_id', 'item_id', 'is_watched']]

In [5]:
train_data['user_id'] = train_data['user_id'].str.replace('user_', '').astype(int)
train_data['item_id'] = train_data['item_id'].str.replace('video_', '').astype(int)

/tmp/ipykernel_485/3264752054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['user_id'] = train_data['user_id'].str.replace('user_', '').astype(int)
/tmp/ipykernel_485/3264752054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['item_id'] = train_data['item_id'].str.replace('video_', '').astype(int)


In [37]:
user_id_dict = dict(enumerate(train_data['user_id'].unique(), 0))
# item_id_dict = train_data['item_id'].to_dict()
user_id_dict = {val: key for key, val in user_id_dict.items()}

In [38]:
len(user_id_dict)

1983158

In [30]:
model.predict(1983157, np.arange(1266073))

array([ 2.6854227 ,  0.90102494,  1.6418545 , ..., -1.4472134 ,
       -1.0879052 , -1.2399035 ], dtype=float32)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
for user_id in submission['user_id'].values:
    user_id = int(user_id.replace('user_', ''))
    try:
        user_id_matrix = user_id_dict[user_id]
        recommendations = model.predict(user_id_matrix, np.arange(1266073))
        top_items = np.argsort(-recommendations)[:1000]
        history = list(OrderedDict.fromkeys(clean_pst_df[clean_pst_df['user_id'] == user_id]['item_id']))
        recommendations = [item_id_dict.get(key) for key in recommendations]
        unique_list = set(OrderedDict.fromkeys(recommendations)) - set(history)
    except KeyError:
        unique_list = ['video_0', 'video_0','video_0','video_0','video_0','video_0','video_0','video_0','video_0','video_0']
    submission[submission['user_id'] == user_id]['recs'] = [f"user_{userid}" for userid in list(unique_list)[:10]]
submission.to_csv("submission.csv", index=False)

/tmp/ipykernel_485/20173477.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission[submission['user_id'] == user_id]['recs'] = [f"user_{userid}" for userid in list(unique_list)[:10]]


In [ ]:
submission.head()

In [ ]:
%time
user_id_matrix = user_id_dict[user_id]
recommendations = model.predict(user_id_matrix, np.arange(1266073))
top_items = np.argsort(-recommendations)[:1000]
history = list(OrderedDict.fromkeys(clean_pst_df[clean_pst_df['user_id'] == user_id]['item_id']))
recommendations = [item_id_dict.get(key) for key in recommendations]
unique_list = set(OrderedDict.fromkeys(recommendations)) - set(history)
[f"user_{userid}" for userid in list(unique_list)[:10]]